In [1]:
#import libraries
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans
from collections import Counter 

#do not show warnings
import warnings
warnings.filterwarnings("ignore")



from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor




In [9]:
sample = pd.read_csv("sample_submission.csv")


In [10]:
sample

,id,target
0,0;133,0
1,0;5,1
2,0;10,0
3,0;396,1
4,0;14,0
...,...,...
790444,19998;26,0
790445,19998;31,0
790446,19998;29,1
790447,19998;798,1


In [11]:
train = pd.read_csv("train.csv")

In [12]:
train

,user_id,order_completed_at,cart
0,2,2015-03-22 09:25:46,399
1,2,2015-03-22 09:25:46,14
2,2,2015-03-22 09:25:46,198
3,2,2015-03-22 09:25:46,88
4,2,2015-03-22 09:25:46,157
...,...,...,...
3123059,12702,2020-09-03 23:45:45,441
3123060,12702,2020-09-03 23:45:45,92
3123061,12702,2020-09-03 23:45:45,431
3123062,12702,2020-09-03 23:45:45,24


In [13]:
train.describe()

In [14]:
train.dtypes

In [15]:
missing_values_count = train.isnull().sum()
missing_values_count

In [16]:
total_cells = np.product(train.shape)
total_missing = missing_values_count.sum()

percent_missing = (total_missing/total_cells) * 100
print(percent_missing)

In [17]:
print(train.user_id.unique())

In [18]:
train.user_id.value_counts().head()

In [19]:
print(train['order_completed_at'].head())

In [20]:
train['order_completed_at_parsed'] = pd.to_datetime(train['order_completed_at'], infer_datetime_format=True)

In [21]:
train['order_completed_at_parsed'].head()

In [22]:
train = train.drop('order_completed_at', 1)

In [23]:
train = train.drop('order_completed_at_parsed', 1)

In [24]:
d = train['cart']

In [17]:
train['target'] = train.duplicated(keep = False)
train['target'] = train["target"].astype(int)
train

NameError: name 'train' is not defined

In [16]:
train = train.loc[train.duplicated(subset=['target'], keep=False)]

NameError: name 'train' is not defined

In [26]:
train.to_csv('train_n.csv')

In [27]:
train.dtypes

user_id    int64
cart       int64
target     int32
dtype: object

In [28]:
train['id'] = train['user_id'].astype(str) +";"+ train['cart'].astype(str)

In [29]:
train = train.drop('user_id', 1)

In [30]:
train

,cart,target,id
0,399,0,2;399
1,14,0,2;14
2,198,1,2;198
3,88,1,2;88
4,157,1,2;157
...,...,...,...
3123059,441,1,12702;441
3123060,92,0,12702;92
3123061,431,1,12702;431
3123062,24,1,12702;24


In [31]:
train = train.drop('cart', 1)

In [32]:
train

,target,id
0,0,2;399
1,0,2;14
2,1,2;198
3,1,2;88
4,1,2;157
...,...,...
3123059,1,12702;441
3123060,0,12702;92
3123061,1,12702;431
3123062,1,12702;24


In [33]:
train_new = train.reindex(columns=['id','target'])

In [4]:
#train_new.to_csv('train_new.csv')

In [2]:
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [3]:
train_new_filepath= 'train_n.csv'
train_new = pd.read_csv(train_new_filepath)
train_new = train_new.drop('Unnamed: 0', 1)

In [18]:
train_new

,user_id,cart,target
0,2,399,0
1,2,14,0
2,2,198,1
3,2,88,1
4,2,157,1
...,...,...,...
3123059,12702,441,1
3123060,12702,92,0
3123061,12702,431,1
3123062,12702,24,1


In [11]:
y = train_new.cart

In [12]:
features = ['target', 'user_id']
X = train_new[features].copy()
X_test = train_new[features].copy()


In [13]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [9]:
from xgboost import XGBRegressor
my_model_1 = XGBRegressor(random_state = 0) 


my_model_1.fit(X_train, y_train) 

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [10]:
from sklearn.metrics import mean_absolute_error


predictions_1 = my_model_1.predict(X_valid)
print("Mean Absolute error: " + str(mean_absolute_error(predictions_1, y_valid)))

Mean Absolute error: 185.06822891178308


In [11]:
mae_1 = mean_absolute_error(predictions_1, y_valid)
print("Mean Absolute Error: ", mae_1) 

Mean Absolute Error:  185.06822891178308


In [12]:
from xgboost import XGBRegressor

my_model_2 = XGBRegressor(n_estimators = 1000, learning_rate = 0.05, random_state = 0)
my_model_2.fit(X_train, y_train)
predictions_2 = my_model_2.predict(X_valid)

mae_2 = mean_absolute_error(predictions_2, y_valid)

In [13]:
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 184.96681378828032


In [ ]:
train_model = DecisionTreeRegressor(random_state=1)
train_model.fit(train_X, train_y)

In [22]:
val_predictions = train_model.predict(val_X)
print(val_predictions)
val_mae=mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying ma_leaf_nodes: {:,.0f}".format(val_mae))

In [23]:
train_model = DecisionTreeRegressor(max_leaf_nodes = 100, random_state=1)
train_model.fit(train_X, train_y)
val_predictions = train_model.predict(val_X)
print(val_predictions)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))

In [24]:
print("Making predictions")
print(X.head())
print("The predictions are")
print(train_model.predict(X.head()))

In [10]:
from sklearn.neighbors import KNeighborsRegressor

In [14]:
KNN_model = KNeighborsRegressor(n_neighbors=3).fit(X_train,y_train)

In [15]:
KNN_predict = KNN_model.predict(X_test)

In [16]:
print(KNN_predict)

[331.66666667 331.66666667 176.         ... 283.66666667 283.66666667
 205.66666667]
